In [ ]:
pip install plotly==5.11.0

In [3]:
# PyTorch package and submodules
import torch
import torch.nn as nn
from torch.optim import SGD 

# NumPy for math operations, and Pandas for processing tabular data.
import numpy as np
import pandas as pd

# Plotly plotting package
import plotly.graph_objects as go
import plotly.express as px

from sklearn import preprocessing
from google.colab.data_table import DataTable

# AUROC and average precision (AP) scores from sklearn
from sklearn.metrics import roc_auc_score, average_precision_score

# Creating a DataFrame from our dataset
df = pd.read_csv('Assignment_1_data_R1.csv')


In [ ]:
df.head(7)

In [ ]:
df.info ()

In [ ]:
df.dtypes

Data cleaning

In [ ]:
# removing NaN values

red_df = df.dropna()
red_df.info()

In [5]:
X_raw = red_df.iloc[:,:33]
y_df = red_df.iloc[:,[33]]


In [ ]:
# Normalize original data first.
X_df = (X_raw-X_raw.mean())/X_raw.std()
X_df.describe()

Making the split

In [7]:
# We use an approx 6:4 train test splitting
np.random.seed(12)
cases = ['train','test']
case_list = np.random.choice(cases,size=len(X_df),replace=True,p=[0.6,0.4])

X_df_RTrain, y_df_RTrain = X_df.iloc[case_list=='train'], y_df.iloc[case_list=='train']
X_df_RTest, y_df_RTest = X_df.iloc[case_list=='test'], y_df.iloc[case_list=='test']

In [8]:
#check split value is correct 
y_df_RTrain.value_counts()

outcome
0.0        9476
1.0         391
dtype: int64

In [201]:
from sklearn.linear_model import LogisticRegression as logit # use build-in logistic regression model in sklearn
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_curve, precision_recall_curve

using SMOTE 

In [9]:
# Use imbalanced learn package
from imblearn.over_sampling import SMOTE

In [10]:
# Use SMOTE to resample minority class.
smote_sampler = SMOTE(random_state=12,sampling_strategy='minority')
X_df_SMOTE, y_df_SMOTE = smote_sampler.fit_resample(X_df_RTrain, y_df_RTrain)

In [11]:
y_df_SMOTE.value_counts()

outcome
0.0        9476
1.0        9476
dtype: int64

Applying logistics regression

In [ ]:
# We convert training and testing dataframe to PyTorch tensor datatype,

X_train = torch.tensor(X_df_SMOTE.to_numpy(),dtype=torch.float32)
m,n = X_train.shape
y_train= torch.tensor(y_df_SMOTE.to_numpy(),dtype=torch.float32).reshape(m,1)


X_test = torch.tensor(X_df_RTest.to_numpy(),dtype=torch.float32)
m,n = X_test.shape
y_test = torch.tensor(y_df_RTest.to_numpy(),dtype=torch.float32).reshape(m,1)


h = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

# Logistic model is linear+sigmoid
f = torch.nn.Sequential(
    h,
    sigma
)

J_BCE = torch.nn.BCELoss()
GD_optimizer = torch.optim.SGD(lr=0.001,params=f.parameters())

nIter = 10000
printInterval = 1000

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f(X_train)
    loss = J_BCE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

In [91]:
# Saving the model parameters for later comparisons

weight = h.weight.detach().squeeze().clone()

In [86]:
# Test on test data

threshold = 0.5

with torch.no_grad():
    pred_test = f(X_test)

# The output has shape M*1. Use .squeeze to remove the trailing dimension with size 1.
binary_pred = np.where(pred_test.squeeze()>threshold,'Malignant','Benign')
label = np.where(y_test.squeeze()>0.5,'Malignant','Benign')
acc = (binary_pred==label).sum()/binary_pred.shape[0]
print('Accuracy on test dataset is {:.2f}%'.format(acc*100))

Accuracy on test dataset is 73.08%


Logistics regression with L1 regularization

In [ ]:
h_L1 = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

# Logistic model is linear+sigmoid
f_L1 = torch.nn.Sequential(
    h_L1,
    sigma
)

J_BCE = torch.nn.BCELoss()

GD_optimizer = torch.optim.Adam(lr=0.01,params=f_L1.parameters())

# Define L_1 regularization
def L1_reg(model,lbd):
    result = torch.tensor(0)
    for param in model.parameters(): # iterate over all parameters of our model
        result = result + param.abs().sum()

    return lbd*result


nIter = 500
printInterval = 50
lbd = 0.03 # L1 reg strength

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f_L1(X_train)
    loss = J_BCE(pred,y_train)
    (loss+L1_reg(f_L1,lbd)).backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f_L1(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

In [95]:
weight_L1 = h_L1.weight.detach().squeeze().clone()

Logistics regression with L2 regularization

In [ ]:
h_L2 = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

# Logistic model is linear+sigmoid
f_L2 = torch.nn.Sequential(
    h_L2,
    sigma
)

J_BCE = torch.nn.BCELoss()

# PyTorch optimizer support L2 regularization by
# setting the weight_decay parameter, which corresponds to
# the regularization strength.
GD_optimizer = torch.optim.Adam(lr=0.01,params=f_L2.parameters(),weight_decay=0.05)

nIter = 500
printInterval = 50

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f_L2(X_train)
    loss = J_BCE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f_L2(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

In [93]:
weight_L2 = h_L2.weight.detach().squeeze().clone()

Visualizing and comparing the weights of the three models

In [97]:
weight_df = pd.DataFrame(
    {
        'vanilla':weight,
        'L2':weight_L2,
        'L1':weight_L1
    }
).melt(id_vars=[],value_vars=['vanilla','L2','L1'])
weight_df


fig = px.box(
    weight_df,
    y='value',
    facet_col='variable',
    color='variable',
    points='all',
    title='Logistic Regression Weights Distributions'
)
fig.update_yaxes(
    matches=None,
    showticklabels=True
)
fig.update_traces(jitter=0.5)

Forward and Backward selection method for features selection

In [98]:
from sklearn.linear_model import LogisticRegression as logit 
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_curve, precision_recall_curve

In [ ]:
model = logit(penalty='l2',C=1/10,solver='saga') 

# Forward feature selection.
forward_selection = SFS(
    model, n_features_to_select=3, direction="forward"
).fit(X_df_SMOTE, y_df_SMOTE)

# Backward feature selection.
backward_selection = SFS(
    model, n_features_to_select=3, direction="backward"
).fit(X_df_SMOTE, y_df_SMOTE)

In [122]:
forward_selection.get_feature_names_out()

array(['gender', 'age', 'heart_rate_min'], dtype=object)

In [123]:
backward_selection.get_feature_names_out()

array(['sofa_cns', 'sofa_renal', 'charlson_comorbidity_index'],
      dtype=object)

Comparison of models - stepwise selection

In [ ]:
# Full model
model.fit(X_df_SMOTE,y_df_SMOTE)
y_pred_full = model.predict_proba(X_df_RTest)

# Model with forward selected features
model.fit(forward_selection.transform(X_df_SMOTE),y_df_SMOTE)
y_pred_FS = model.predict_proba(forward_selection.transform(X_df_RTest))

# Model with backward selected features
model.fit(backward_selection.transform(X_df_SMOTE),y_df_SMOTE)
y_pred_BS = model.predict_proba(backward_selection.transform(X_df_RTest))

ROC curves

In [110]:
# roc_curve
fpr_full, tpr_full, _ = roc_curve(y_df_RTest,y_pred_full[:,1])
fpr_FS, tpr_FS, _ = roc_curve(y_df_RTest,y_pred_FS[:,1])
fpr_BS, tpr_BS, _ = roc_curve(y_df_RTest,y_pred_BS[:,1])

roc_df = pd.DataFrame(
    {
        'False Positive Rate':np.hstack([fpr_full,fpr_FS,fpr_BS]),
        'True Positive Rate':np.hstack([tpr_full,tpr_FS,tpr_BS]),
        'method':['full_model']*len(fpr_full)+['FS']*len(fpr_FS)+['BS']*len(fpr_BS)
    }
)

In [111]:
# Visualize ROC curve
fig = px.line(roc_df,y='True Positive Rate',x='False Positive Rate',facet_col='method',color='method')
fig

Precision recall curves 

In [112]:
# precision recall curves
p_full, r_full, _ = precision_recall_curve(y_df_RTest,y_pred_full[:,1])
p_FS, r_FS, _ = precision_recall_curve(y_df_RTest,y_pred_FS[:,1])
p_BS, r_BS, _ = precision_recall_curve(y_df_RTest,y_pred_BS[:,1])

pr_df = pd.DataFrame(
    {
        'Precision':np.hstack([p_full,p_FS,p_BS]),
        'Recall':np.hstack([r_full,r_FS,r_BS]),
        'method':['Full Model']*len(p_full)+['Forward Selection']*len(p_FS)+['Backward Selection']*len(p_BS)
    }
)

In [113]:
# Visualize precision recall curve
fig = px.line(pr_df,x='Recall',y='Precision',facet_col='method',color='method')
fig

AUROC and AP score

In [117]:
auroc_full = roc_auc_score(y_df_RTest,y_pred_full[:,1])
auroc_FS = roc_auc_score(y_df_RTest,y_pred_FS[:,1])
auroc_BS = roc_auc_score(y_df_RTest,y_pred_BS[:,1])

ap_full = average_precision_score(y_df_RTest,y_pred_full[:,1])
ap_FS = average_precision_score(y_df_RTest,y_pred_FS[:,1])
ap_BS = average_precision_score(y_df_RTest,y_pred_BS[:,1])

print('Full feature dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc_full,ap_full))
print('FS feature dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc_FS,ap_FS))
print('BS feature dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc_BS,ap_BS))

Full feature dataset: AUROC 0.753, AP 0.141
FS feature dataset: AUROC 0.760, AP 0.141
BS feature dataset: AUROC 0.748, AP 0.133


Tree Based Models

In [16]:
# Plotly plotting package
import plotly.graph_objects as go
import plotly.express as px

# Tools in sklearn to select best model
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV

# Decision tree classifier in sklearn
from sklearn.tree import DecisionTreeClassifier as DTC, plot_tree

# We use f1 score to test model performance
from sklearn.metrics import f1_score

# Import matplotlib.pyplot to visualize tree models
import matplotlib.pyplot as plt

In [176]:
# Build a decision tree.
TreeModel = DTC(criterion='entropy',max_depth=3,random_state=15)
TreeModel.fit(X_df_SMOTE,y_df_SMOTE)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=15)

In [ ]:
# Splitting rules can be visualized by using plot_tree in sklearn
c_names = list(X_df_SMOTE.columns)

plt.figure(figsize=(14,10))
plot_tree(
    TreeModel,
    filled=True,
    feature_names = c_names
    #class_names = ['no intubate','intubate']
)

In [178]:
# Using `GridSearchCV` to select the best `max_depth`.

parameters = {'max_depth':np.arange(start=1,stop=10,step=1)}
stratifiedCV = StratifiedKFold(n_splits=8)
TreeModel = DTC(criterion='entropy')
BestTree = GridSearchCV(
    TreeModel,
    param_grid=parameters,
    scoring='average_precision',
    cv=stratifiedCV
)
BestTree.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=8, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(criterion='entropy'),
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='average_precision')

In [179]:
BestTree.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=9)

In [180]:
BestTree.best_score_

0.9542178594187548

In [ ]:
y_pred = BestTree.predict(X_df_RTest)

print('AP score on test set: {:.4f}'.format(average_precision_score(y_df_RTest,y_pred)))
pd.crosstab(y_df_RTest["outcome"],y_pred)

XGBoost

In [ ]:
from xgboost import XGBClassifier as XGBC

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
stratifiedCV = StratifiedKFold(n_splits=8)
# XGBC: XGBoost classifier
XGBoostModel = XGBC()
BestXGBoost = GridSearchCV(
    XGBoostModel,
    param_grid=parameters,
    scoring='average_precision',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1 # use all cpu cores to speedup grid search
)
BestXGBoost.fit(X_df_SMOTE,np.ravel(y_df_SMOTE))

In [ ]:
BestXGBoost.best_params_

In [ ]:
BestXGBoost.best_score_

In [ ]:
y_pred = BestXGBoost.predict(X_df_RTest)
print('AP score on test set: {:.3f}'.format(average_precision_score(y_df_RTest,y_pred)))
pd.crosstab(y_df_RTest["outcome"],y_pred)

Linear SVM

In [12]:
from sklearn.svm import SVC

In [ ]:
# 'C': strength of L2 regularization on linear SVM. Larger 'C' --> smaller regularization.
parameters = {
    'C':np.arange(start=1,stop=20,step=5)
}
stratifiedCV = StratifiedKFold(n_splits=8)
SVCModel = SVC(kernel='linear')
BestSVC = GridSearchCV(
    SVCModel,
    param_grid=parameters,
    scoring='average_precision',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)
BestSVC.fit(X_df_SMOTE,np.ravel(y_df_SMOTE))

In [ ]:
BestSVC.best_estimator_

In [ ]:
BestSVC.best_score_

In [ ]:
y_pred = BestSVC.predict(X_df_RTest)
print('AP score on test set: {:.4f}'.format(f1_score(y_df_RTest,y_pred)))
pd.crosstab(y_df_RTest,y_pred)

Non-Linear SVM

In [ ]:
parameters = {
    'C':np.arange(start=0.5,stop=5,step=0.5)
}
stratifiedCV = StratifiedKFold(n_splits=8)
SVMrbfModel = SVC(kernel='rbf')
BestSVMrbf = GridSearchCV(
    SVMrbfModel,
    param_grid=parameters,
    scoring='average_precision',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)
BestSVMrbf.fit(X_df_SMOTE,np.ravel(y_df_SMOTE))


In [ ]:
BestSVMrbf.best_estimator_

In [ ]:
BestSVMrbf.best_score_

In [ ]:
y_pred = BestSVMrbf.predict(X_df_RTest)
print('AP score on test set: {:.4f}'.format(average_precision_score(y_df_RTest,y_pred)))
pd.crosstab(y_df_RTest["outcome"],y_pred)